In [ ]:
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['D1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]
    Src['Datetime']                     = Query['datetime']
    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    # Src['Change']   = (Src['Close'] - Src['Open'])
    # Src['Hilo']     = (Src['High']  - Src['Low'])
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Open -1']     = Calc['Open']  .shift(+1)
    Calc['High -1']     = Calc['High']  .shift(+1)
    Calc['Low -1']      = Calc['Low']   .shift(+1)
    Calc['Close -1']    = Calc['Close'] .shift(+1)
    
    Calc['Change -1']   = (Calc['Close -1'] - Calc['Open -1'])
    Calc['Hilo -1']     = (Calc['High -1']  - Calc['Low -1'])
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

In [ ]:
BINS = [500,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500,6000,6500,7000]

In [ ]:
def SIGNALS(Calc):
    Sign = pd.DataFrame(Calc)
    Sign['True'] = True

    Sign['Chg -1 Is Bull']  = (Calc['Change -1'] >= 0)
    Sign['Chg -1 Is Bear']  = (Calc['Change -1'] <  0)

    for x in BINS:
        Sign[f'Hilo -1 > {x}'] = (Calc['Hilo -1'] > x) 

    for OHLC in ['Open','High','Low','Close']:
        Sign[f'{OHLC} out Max'] =                                                 (              Calc['High -1'] < Calc[OHLC])
        Sign[f'{OHLC} in Max']  =                                                 (Calc[OHLC] <= Calc['High -1']             )
        Sign[f'{OHLC} inside']  = (              Calc['Low -1'] < Calc[OHLC])  &  (Calc[OHLC] <= Calc['High -1']             )
        Sign[f'{OHLC} in Min']  = (              Calc['Low -1'] < Calc[OHLC])
        Sign[f'{OHLC} out Min'] = (Calc[OHLC] <= Calc['Low -1']             )

    Sign['Break 0'] = (Sign['Open in Min']  & Sign['Open in Max'])
    Sign['Break 1'] = (Sign['Open out Min'] | Sign['Open out Max'])  
    return Sign

Sign = SIGNALS(Calc)
Sign.head()

# Stats

In [ ]:
def STATS(Sign, BINS, conds):
    pipe = []
    for _bin in BINS:
        Df = Sign[Sign[_bin] & Sign[conds].all(1)] 
        pipe.append({
            'Condition':    _bin,
            'Count':        len(Df),
            'Inside':       np.round(Df['Break 0'].mean() *100, 0).astype(int), 
            'Outside':      np.round(Df['Break 1'].mean() *100, 0).astype(int), 
        })
    pass
    return pd.DataFrame(pipe)

In [126]:
STATS(Sign, BINS=Sign.loc[:, 'Hilo -1 > 500':'Hilo -1 > 6500'].columns, conds=['True'])

,Condition,Count,Inside,Outside
0,Hilo -1 > 500,1244,78,22
1,Hilo -1 > 1000,1180,79,21
2,Hilo -1 > 1500,863,79,21
3,Hilo -1 > 2000,492,78,22
4,Hilo -1 > 2500,258,80,20
5,Hilo -1 > 3000,140,82,18
6,Hilo -1 > 3500,59,83,17
7,Hilo -1 > 4000,31,81,19
8,Hilo -1 > 4500,15,93,7
9,Hilo -1 > 5000,6,83,17


In [125]:
STATS(Sign, BINS=Sign.loc[:, 'Hilo -1 > 500':'Hilo -1 > 6500'].columns, conds=['Chg -1 Is Bull'])

,Condition,Count,Inside,Outside
0,Hilo -1 > 500,642,77,23
1,Hilo -1 > 1000,604,78,22
2,Hilo -1 > 1500,432,79,21
3,Hilo -1 > 2000,231,79,21
4,Hilo -1 > 2500,114,80,20
5,Hilo -1 > 3000,60,83,17
6,Hilo -1 > 3500,27,78,22
7,Hilo -1 > 4000,12,67,33
8,Hilo -1 > 4500,6,83,17
9,Hilo -1 > 5000,3,67,33


In [124]:
STATS(Sign, BINS=Sign.loc[:, 'Hilo -1 > 500':'Hilo -1 > 5500'].columns, conds=['Chg -1 Is Bear'])

,Condition,Count,Inside,Outside
0,Hilo -1 > 500,602,79,21
1,Hilo -1 > 1000,576,80,20
2,Hilo -1 > 1500,431,80,20
3,Hilo -1 > 2000,261,77,23
4,Hilo -1 > 2500,144,80,20
5,Hilo -1 > 3000,80,81,19
6,Hilo -1 > 3500,32,88,12
7,Hilo -1 > 4000,19,89,11
8,Hilo -1 > 4500,9,100,0
9,Hilo -1 > 5000,3,100,0


# Snippets

In [ ]:
def SIGNALS(Calc):
    Sign = pd.DataFrame(Calc)
    Sign['True'] = True

    Sign['Chg -1 Is Bull']  = (Calc['Change -1'] >= 0)
    Sign['Chg -1 Is Bear']  = (Calc['Change -1'] <  0)

    for x in BINS:
        Sign[f'Hilo -1 > {x}'] = (Calc['Hilo -1'] > x) 

    for OHLC in ['Open','High','Low','Close']:
        Sign[f'{OHLC} out Max'] =                                                 (              Calc['High -1'] < Calc[OHLC])
        Sign[f'{OHLC} in Max']  =                                                 (Calc[OHLC] <= Calc['High -1']             )
        Sign[f'{OHLC} inside']  = (              Calc['Low -1'] < Calc[OHLC])  &  (Calc[OHLC] <= Calc['High -1']             )
        Sign[f'{OHLC} in Min']  = (              Calc['Low -1'] < Calc[OHLC])
        Sign[f'{OHLC} out Min'] = (Calc[OHLC] <= Calc['Low -1']             )

    Sign['Break 0'] = (Sign['Open in Min']  & Sign['Open in Max'])  
    Sign['Break 1'] = (Sign['Open out Min'] | Sign['Open out Max'])  
    return Sign



def STATS(Sign, BINS, conds):
    pipe = []
    for _bin in BINS:
        Df = Sign[Sign[_bin] & Sign[conds].all(1)] 
        pipe.append({
            'Condition':    _bin,
            'Count':        len(Df),
            'Inside':       np.round(Df['Break 0'].mean() *100, 0).astype(int), 
            'Outside':      np.round(Df['Break 1'].mean() *100, 0).astype(int), 
        })
    pass
    return pd.DataFrame(pipe)


STATS(Sign, BINS=Sign.loc[:, 'Hilo -1 > 500':'Hilo -1 > 6500'].columns, conds=['True'])
STATS(Sign, BINS=Sign.loc[:, 'Hilo -1 > 500':'Hilo -1 > 6500'].columns, conds=['Chg -1 Is Bull'])
STATS(Sign, BINS=Sign.loc[:, 'Hilo -1 > 500':'Hilo -1 > 5500'].columns, conds=['Chg -1 Is Bear'])